# Patch-based U-Net implementation on Lung-Segmented or Preprocessed Scans

Nanyan "Rosalie" Zhu & Chen "Raphael" Liu

$(1)$ We have two sets of data available upon completion of the preprocessing steps. The first set is the lung-segmented or preprocessed scans that will be treated as the input to our U-Net. The second set is the ground truth annotation scans that will be treated as the expected output of our U-Net.

$(2)$ Our baseline model is a U-Net that takes single horizontal slices from the lung-segmented or preprocessed scans as inputs and predicts the corresponding single slices of ground truth scans.

$(3)$ Several things to keep in mind on determining the train. validation and test sets:
    - Slices from the same scan has to be categorized in no more than one of such classes.
    - After the scans are assigned to their designated classes, they can then be randomized on slice level and put into batches.

### Import packages

In [1]:
import argparse
import os
from solver import Solver
from data_loader import get_loader, prediction_loader
from torch.backends import cudnn
import random
import numpy as np
from glob import glob as glob
import matplotlib.pyplot as plt

### Refer to **scans_to_images.ipynb** to fill in the following hyperparamters.

In [2]:
majority_voting_threshold = 1 # We need more than 1 radiologist to confirm.
augmentation_rate = 5 # 5 patches on one single claimed (x, y, z) nodule center.
negative_patch_ratio = 0.25 # This will result in an overall positve-negative ratio of 80 % to 20 %.

In [3]:
dataset_path = './dataset/Patches_with_nodules-' + '%d-%d-%.4f'%(majority_voting_threshold, augmentation_rate, negative_patch_ratio)

### Setup parameters (will become input parsers in the .py version).

In [4]:
class config():   
    # model hyper-parameters
    image_size= int(32)
    t = int(3) # t for Recurrent step of R2U_Net or R2AttU_Net'
    img_ch = int(1)
    output_ch = int(1)
    num_epochs = int(200)
    batch_size = int(1)
    num_workers = int(1)
    lr = float(0.001)
    
    optimizer_choice = 'SGD'
    
    if optimizer_choice == 'Adam':
        beta1 = float(0.5) # momentum1 in Adam
        beta2 = float(0.999) # momentum2 in Adam
    elif optimizer_choice == 'SGD':
        momentum = float(0.9)
    else:
        print('No such optimizer available')
    
    augmentation_prob = float(0)
    
    PPorLS = 'LS' # Use preprocessed scans or lung-segmented scans for training, validating and testing.

    log_step = int(2)
    val_step = int(2)
    # validation frequency within epoch, [first epoch, other epoches]
    val_freq_batch = [2000, 8000]

    # misc
    mode = 'train'
    model_type = 'U_Net' # 'U_Net/R2U_Net/AttU_Net/R2AttU_Net'
    model_path = './models/' + PPorLS + '_%d-%d-%.4f' % (majority_voting_threshold, augmentation_rate, negative_patch_ratio) + \
    '/' + str(model_type) + '/' + str(optimizer_choice) + '/'
    
    if PPorLS == 'LS':
        train_path = dataset_path + '/train/'
        valid_path = dataset_path + '/valid/'
        test_path = dataset_path + '/test/'
        result_path = './results/LS_' + '%d-%d-%.4f' % (majority_voting_threshold, augmentation_rate, negative_patch_ratio) + \
        '/' + str(model_type) + '/' + str(optimizer_choice) + '/'
        result_img_path = './results/LS_' + '%d-%d-%.4f' % (majority_voting_threshold, augmentation_rate, negative_patch_ratio) + \
        '/' + str(model_type) + '/' + str(optimizer_choice) + '/img/'
    elif PPorLS == 'PP':
        train_path = dataset_path + '/train_PP/'
        valid_path = dataset_path + '/valid_PP/'
        test_path = dataset_path + '/test_PP/'
        result_path = './results/PP_' + '%d-%d-%.4f' % (majority_voting_threshold, augmentation_rate, negative_patch_ratio) + \
        '/' + str(model_type) + '/' + str(optimizer_choice) + '/'
        result_img_path = './results/PP_' + '%d-%d-%.4f' % (majority_voting_threshold, augmentation_rate, negative_patch_ratio) + \
        '/' + str(model_type) + '/' + str(optimizer_choice) + '/img/'

    whole_slices_path = './dataset/All_slices_with_nodules/test/1.3.6.1.4.1.14519.5.2.1.6279.6001.185154482385982570363528682299_segmented_167.png'
    
    cuda_idx = int(1)

In [5]:
cudnn.benchmark = True

In [6]:
if config.model_type not in ['U_Net','R2U_Net','AttU_Net','R2AttU_Net']:
    print('ERROR!! model_type should be selected in U_Net/R2U_Net/AttU_Net/R2AttU_Net')
    print('Your input for model_type was %s'%model_type)

### Create directories if not exist

In [7]:
if not os.path.exists(config.model_path):
    os.makedirs(config.model_path)

if not os.path.exists(config.result_path):
    os.makedirs(config.result_path)

if not os.path.exists(config.result_img_path):
    os.makedirs(config.result_img_path)

### More parameters

In [8]:
augmentation_prob = 0
config.augmentation_prob = augmentation_prob
vars(config)

mappingproxy({'__module__': '__main__',
              'image_size': 32,
              't': 3,
              'img_ch': 1,
              'output_ch': 1,
              'num_epochs': 200,
              'batch_size': 1,
              'num_workers': 1,
              'lr': 0.001,
              'optimizer_choice': 'SGD',
              'momentum': 0.9,
              'augmentation_prob': 0,
              'PPorLS': 'LS',
              'log_step': 2,
              'val_step': 2,
              'val_freq_batch': [2000, 8000],
              'mode': 'train',
              'model_type': 'U_Net',
              'model_path': './models/LS_1-5-0.2500/U_Net/SGD/',
              'train_path': './dataset/Patches_with_nodules-1-5-0.2500/train/',
              'valid_path': './dataset/Patches_with_nodules-1-5-0.2500/valid/',
              'test_path': './dataset/Patches_with_nodules-1-5-0.2500/test/',
              'result_path': './results/LS_1-5-0.2500/U_Net/SGD/',
              'result_img_path': './results/

### Initialize the loaders

In [9]:
train_loader = get_loader(image_path = config.train_path,
                        image_size = config.image_size,
                        batch_size = config.batch_size,
                        num_workers = config.num_workers,
                        mode = 'train',
                        augmentation_prob = config.augmentation_prob,
                        PPorLS = config.PPorLS)

valid_loader = get_loader(image_path = config.valid_path,
                        image_size = config.image_size,
                        batch_size = config.batch_size,
                        num_workers = config.num_workers,
                        mode = 'valid',
                        augmentation_prob = 0.,
                        PPorLS = config.PPorLS)

test_loader = get_loader(image_path = config.test_path,
                        image_size = config.image_size,
                        batch_size = config.batch_size,
                        num_workers = config.num_workers,
                        mode = 'test',
                        augmentation_prob = 0.,
                        shuffle = False,
                        PPorLS = config.PPorLS)

whole_slice_prediction_loader = prediction_loader(image_path = config.whole_slices_path)

image count in train path :23647
image count in valid path :3737
image count in test path :3002


### Call the solver

In [10]:
solver = Solver(config, train_loader, valid_loader, test_loader, whole_slice_prediction_loader)

In [ ]:
# Train and sample the images
if config.mode == 'train':
    train_loss_history, validation_loss_history, val_batch_loss_history = solver.train()
elif config.mode == 'test':
    solver.test()

## THE ONES TO BE RECORDED (problematic scans)

In [135]:
filename,slice_num, x_num, y_num

('1.3.6.1.4.1.14519.5.2.1.6279.6001.306948744223170422945185006551',
 '115',
 '297',
 '168')

In [139]:
filename,slice_num, x_num, y_num

('1.3.6.1.4.1.14519.5.2.1.6279.6001.133378195429627807109985347209',
 '047',
 '157',
 '240')

In [141]:
filename,slice_num, x_num, y_num

('1.3.6.1.4.1.14519.5.2.1.6279.6001.287966244644280690737019247886',
 '099',
 '314',
 '275')

In [145]:
filename,slice_num, x_num, y_num

('1.3.6.1.4.1.14519.5.2.1.6279.6001.131939324905446238286154504249',
 '160',
 '092',
 '275')

In [151]:
filename,slice_num, x_num, y_num

('1.3.6.1.4.1.14519.5.2.1.6279.6001.183982839679953938397312236359',
 '223',
 '135',
 '254')

In [153]:
filename,slice_num, x_num, y_num

('1.3.6.1.4.1.14519.5.2.1.6279.6001.202811684116768680758082619196',
 '243',
 '154',
 '228')

In [162]:
filename,slice_num, x_num, y_num

('1.3.6.1.4.1.14519.5.2.1.6279.6001.242761658169703141430370511586',
 '080',
 '082',
 '263')

In [ ]:
R2U_Net_LS

## Our crazy for loop. To choose the "Best" model
model trial: epochs = 30

In [12]:
config.num_epochs = int(30)
config.optimizer_choice = 'SGD'
for config.model_type in ['R2AttU_Net', 'AttU_Net', 'U_Net', 'R2U_Net']:
    if config.optimizer_choice == 'Adam':
        config.beta1 = float(0.5) # momentum1 in Adam
        config.beta2 = float(0.999) # momentum2 in Adam
    elif config.optimizer_choice == 'SGD':
        config.momentum = float(0.9)
    else:
        print('No such optimizer available')
    for config.lr in [float(0.002)]:
        for config.PPorLS in ['LS', 'PP']:
            config.model_path = './models/' + config.PPorLS + '_%d-%d-%.4f' % (majority_voting_threshold, augmentation_rate, negative_patch_ratio) + \
            '/' + str(config.model_type) + '/' + str(config.optimizer_choice) + '/'
            if config.PPorLS == 'LS':
                config.train_path = dataset_path + '/train/'
                config.valid_path = dataset_path + '/valid/'
                config.test_path = dataset_path + '/test/'
                config.result_path = './results/LS_' + '%d-%d-%.4f' % (majority_voting_threshold, augmentation_rate, negative_patch_ratio) + \
                '/' + str(config.model_type) + '/' + str(config.optimizer_choice) + '/'
                config.result_img_path = './results/LS_' + '%d-%d-%.4f' % (majority_voting_threshold, augmentation_rate, negative_patch_ratio) + \
                '/' + str(config.model_type) + '/' + str(config.optimizer_choice) + '/img/'
            elif config.PPorLS == 'PP':
                config.train_path = dataset_path + '/train_PP/'
                config.valid_path = dataset_path + '/valid_PP/'
                config.test_path = dataset_path + '/test_PP/'
                config.result_path = './results/PP_' + '%d-%d-%.4f' % (majority_voting_threshold, augmentation_rate, negative_patch_ratio) + \
                '/' + str(config.model_type) + '/' + str(config.optimizer_choice) + '/'
                config.result_img_path = './results/PP_' + '%d-%d-%.4f' % (majority_voting_threshold, augmentation_rate, negative_patch_ratio) + \
                '/' + str(config.model_type) + '/' + str(config.optimizer_choice) + '/img/'
            cudnn.benchmark = True
            if not os.path.exists(config.model_path):
                os.makedirs(config.model_path)
            if not os.path.exists(config.result_path):
                os.makedirs(config.result_path)
            if not os.path.exists(config.result_img_path):
                os.makedirs(config.result_img_path)
            train_loader = get_loader(image_path = config.train_path,
                    image_size = config.image_size,
                    batch_size = config.batch_size,
                    num_workers = config.num_workers,
                    mode = 'train',
                    augmentation_prob = config.augmentation_prob,
                    PPorLS = config.PPorLS)
            valid_loader = get_loader(image_path = config.valid_path,
                    image_size = config.image_size,
                    batch_size = config.batch_size,
                    num_workers = config.num_workers,
                    mode = 'valid',
                    augmentation_prob = 0.,
                    PPorLS = config.PPorLS)
            test_loader = get_loader(image_path = config.test_path,
                    image_size = config.image_size,
                    batch_size = config.batch_size,
                    num_workers = config.num_workers,
                    mode = 'test',
                    augmentation_prob = 0.,
                    shuffle = False,
                    PPorLS = config.PPorLS)
            whole_slice_prediction_loader = prediction_loader(image_path = config.whole_slices_path)
            solver = Solver(config, train_loader, valid_loader, test_loader, whole_slice_prediction_loader)

            train_loss_history, validation_loss_history, val_batch_loss_history, train_batch_loss_history = solver.train()
            np.save(config.result_path + 'train_loss_history-%.4f.npy' % config.lr, train_loss_history)
            np.save(config.result_path + 'validation_loss_history-%.4f.npy' % config.lr,validation_loss_history)
            np.save(config.result_path + 'validation_batch_loss_history-%.4f.npy' % config.lr, val_batch_loss_history)
            np.save(config.result_path + 'train_batch_loss_history-%.4f.npy' % config.lr, train_batch_loss_history)

image count in train path :23647
image count in valid path :3737
image count in test path :3002
The U-Net path is ./models/LS_1-5-0.2500/R2U_Net/SGD/R2U_Net-30-0.0020-0.0000-LS-SGD.pkl
Start training. The initial learning rate is: 0.002


/home/raphael/anaconda3/envs/LUNGpy36/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


current batch: 0
Current learning rate: 0.002
Current Batch [1] 
[Validation] Validation Loss: -311.2463, Acc: 0.2528, SE: 1.0000, SP: 0.0000, PC: 0.2528, F1: 0.2907, JS: 0.2528, DC: 0.2914, DC_RR: 0.1454
Best R2U_Net model score : 0.2914
current batch: 2000
Current learning rate: 0.002
Current Batch [2001] 
[Validation] Validation Loss: -677.5044, Acc: 0.6548, SE: 0.7358, SP: 0.4681, PC: 0.3186, F1: 0.3426, JS: 0.2781, DC: 0.3487, DC_RR: 0.4427
Best R2U_Net model score : 0.3487
current batch: 4000
Current learning rate: 0.002
Current Batch [4001] 
[Validation] Validation Loss: -895.8268, Acc: 0.2698, SE: 0.6654, SP: 0.2052, PC: 0.1134, F1: 0.1346, JS: 0.0913, DC: 0.1369, DC_RR: 0.2118
current batch: 6000
Current learning rate: 0.002
Current Batch [6001] 
[Validation] Validation Loss: -1325.6120, Acc: 0.6885, SE: 0.2916, SP: 0.7036, PC: 0.1973, F1: 0.1670, JS: 0.1197, DC: 0.1875, DC_RR: 0.4454
current batch: 8000
Current learning rate: 0.002
Current Batch [8001] 
[Validation] Validatio

/home/raphael/anaconda3/envs/LUNGpy36/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


current batch: 0
Current learning rate: 0.002
Current Batch [1] 
[Validation] Validation Loss: -308.6447, Acc: 0.2730, SE: 0.9885, SP: 0.0287, PC: 0.2544, F1: 0.2898, JS: 0.2478, DC: 0.2905, DC_RR: 0.1723
Best R2U_Net model score : 0.2905
current batch: 2000
Current learning rate: 0.002
Current Batch [2001] 
[Validation] Validation Loss: -1255.1527, Acc: 0.7201, SE: 0.4303, SP: 0.7119, PC: 0.3874, F1: 0.3023, JS: 0.2222, DC: 0.3112, DC_RR: 0.5193
Best R2U_Net model score : 0.3112
current batch: 4000
Current learning rate: 0.002
Current Batch [4001] 
[Validation] Validation Loss: -1117.2558, Acc: 0.7090, SE: 0.3829, SP: 0.6801, PC: 0.3092, F1: 0.2627, JS: 0.1947, DC: 0.2750, DC_RR: 0.4888
current batch: 6000
Current learning rate: 0.002
Current Batch [6001] 
[Validation] Validation Loss: -991.3775, Acc: 0.6633, SE: 0.5166, SP: 0.5951, PC: 0.3049, F1: 0.2846, JS: 0.2104, DC: 0.2923, DC_RR: 0.4701
Validation loss stop decreasing. Adjust the learning rate to 0.001.
current batch: 8000
Curr

### Based on our other visualization results, AttU_Net is the best model
result_checking.py and result_checking_inverserate.py for result checking 

epoch = 60

In [9]:
config.num_epochs = int(60)
config.optimizer_choice = 'SGD'
config.model_type = 'AttU_Net'
for config.inverse_ratio in [float(0.1), float(0.25), float(0.5), float(0.75), float(1)]:
    if config.optimizer_choice == 'Adam':
        config.beta1 = float(0.5) # momentum1 in Adam
        config.beta2 = float(0.999) # momentum2 in Adam
    elif config.optimizer_choice == 'SGD':
        config.momentum = float(0.9)
    else:
        print('No such optimizer available')
    for config.lr in [float(0.002)]:
        for config.PPorLS in ['LS', 'PP']:
            config.model_path = './models/' + config.PPorLS + '_%d-%d-%.4f' % (majority_voting_threshold, augmentation_rate, negative_patch_ratio) + \
            '/' + str(config.model_type) + '/' + str(config.optimizer_choice) + '/'
            if config.PPorLS == 'LS':
                config.train_path = dataset_path + '/train/'
                config.valid_path = dataset_path + '/valid/'
                config.test_path = dataset_path + '/test/'
                config.result_path = './results/LS_' + '%d-%d-%.4f' % (majority_voting_threshold, augmentation_rate, negative_patch_ratio) + \
                '/' + str(config.model_type) + '/' + str(config.optimizer_choice) + '/' + str(config.inverse_ratio) + '/'
                config.result_img_path = './results/LS_' + '%d-%d-%.4f' % (majority_voting_threshold, augmentation_rate, negative_patch_ratio) + \
                '/' + str(config.model_type) + '/' + str(config.optimizer_choice) + '/img/'+ str(config.inverse_ratio) + '/'
            elif config.PPorLS == 'PP':
                config.train_path = dataset_path + '/train_PP/'
                config.valid_path = dataset_path + '/valid_PP/'
                config.test_path = dataset_path + '/test_PP/'
                config.result_path = './results/PP_' + '%d-%d-%.4f' % (majority_voting_threshold, augmentation_rate, negative_patch_ratio) + \
                '/' + str(config.model_type) + '/' + str(config.optimizer_choice) + '/'+ str(config.inverse_ratio) + '/'
                config.result_img_path = './results/PP_' + '%d-%d-%.4f' % (majority_voting_threshold, augmentation_rate, negative_patch_ratio) + \
                '/' + str(config.model_type) + '/' + str(config.optimizer_choice) + '/img/'+ str(config.inverse_ratio) + '/'
            cudnn.benchmark = True
            if not os.path.exists(config.model_path):
                os.makedirs(config.model_path)
            if not os.path.exists(config.result_path):
                os.makedirs(config.result_path)
            if not os.path.exists(config.result_img_path):
                os.makedirs(config.result_img_path)
            train_loader = get_loader(image_path = config.train_path,
                    image_size = config.image_size,
                    batch_size = config.batch_size,
                    num_workers = config.num_workers,
                    mode = 'train',
                    augmentation_prob = config.augmentation_prob,
                    PPorLS = config.PPorLS)
            valid_loader = get_loader(image_path = config.valid_path,
                    image_size = config.image_size,
                    batch_size = config.batch_size,
                    num_workers = config.num_workers,
                    mode = 'valid',
                    augmentation_prob = 0.,
                    PPorLS = config.PPorLS)
            test_loader = get_loader(image_path = config.test_path,
                    image_size = config.image_size,
                    batch_size = config.batch_size,
                    num_workers = config.num_workers,
                    mode = 'test',
                    augmentation_prob = 0.,
                    shuffle = False,
                    PPorLS = config.PPorLS)
            whole_slice_prediction_loader = prediction_loader(image_path = config.whole_slices_path)
            solver = Solver(config, train_loader, valid_loader, test_loader, whole_slice_prediction_loader)

            train_loss_history, validation_loss_history, val_batch_loss_history, train_batch_loss_history = solver.train()
            np.save(config.result_path + 'train_loss_history-%.4f.npy' % config.lr, train_loss_history)
            np.save(config.result_path + 'validation_loss_history-%.4f.npy' % config.lr,validation_loss_history)
            np.save(config.result_path + 'validation_batch_loss_history-%.4f.npy' % config.lr, val_batch_loss_history)
            np.save(config.result_path + 'train_batch_loss_history-%.4f.npy' % config.lr, train_batch_loss_history)

image count in train path :23647
image count in valid path :3737
image count in test path :3002
The U-Net path is ./models/LS_1-5-0.2500/AttU_Net/SGD/AttU_Net-60-0.0020-0.0000-LS-SGD-0.1000.pkl
Start training. The initial learning rate is: 0.002


/home/raphael/anaconda3/envs/LUNGpy36/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


current batch: 0
Current learning rate: 0.002
Current Batch [1] 
[Validation] Validation Loss: -302.5845, Acc: 0.2528, SE: 1.0000, SP: 0.0000, PC: 0.2528, F1: 0.2907, JS: 0.2528, DC: 0.2914, DC_RR: 0.2643
Best AttU_Net model score : 0.2643
current batch: 2000
Current learning rate: 0.002
Current Batch [2001] 
[Validation] Validation Loss: -1068.6218, Acc: 0.7233, SE: 0.3792, SP: 0.7417, PC: 0.3158, F1: 0.2615, JS: 0.1810, DC: 0.2696, DC_RR: 0.3061
Best AttU_Net model score : 0.3061
current batch: 4000
Current learning rate: 0.002
Current Batch [4001] 
[Validation] Validation Loss: -779.8170, Acc: 0.7418, SE: 0.0131, SP: 0.7870, PC: 0.0386, F1: 0.0159, JS: 0.0108, DC: 0.0480, DC_RR: 0.0841
current batch: 6000
Current learning rate: 0.002
Current Batch [6001] 
[Validation] Validation Loss: -1464.3750, Acc: 0.7424, SE: 0.2003, SP: 0.7685, PC: 0.2896, F1: 0.2032, JS: 0.1561, DC: 0.2276, DC_RR: 0.2541
current batch: 8000
Current learning rate: 0.002
Current Batch [8001] 
[Validation] Valida

/home/raphael/anaconda3/envs/LUNGpy36/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


current batch: 0
Current learning rate: 0.002
Current Batch [1] 
[Validation] Validation Loss: -301.2619, Acc: 0.7472, SE: 0.0000, SP: 0.7966, PC: 0.0000, F1: 0.0000, JS: 0.0000, DC: 0.0349, DC_RR: 0.0700
Best AttU_Net model score : 0.0700
current batch: 2000
Current learning rate: 0.002
Current Batch [2001] 
[Validation] Validation Loss: -1181.2502, Acc: 0.7437, SE: 0.3539, SP: 0.7533, PC: 0.3475, F1: 0.2935, JS: 0.2321, DC: 0.3105, DC_RR: 0.3359
Best AttU_Net model score : 0.3359
current batch: 4000
Current learning rate: 0.002
Current Batch [4001] 
[Validation] Validation Loss: -1450.5708, Acc: 0.7232, SE: 0.4696, SP: 0.7116, PC: 0.3846, F1: 0.3287, JS: 0.2467, DC: 0.3393, DC_RR: 0.3659
Best AttU_Net model score : 0.3659
current batch: 6000
Current learning rate: 0.002
Current Batch [6001] 
[Validation] Validation Loss: -1502.2516, Acc: 0.7608, SE: 0.2041, SP: 0.7914, PC: 0.2746, F1: 0.2098, JS: 0.1704, DC: 0.2372, DC_RR: 0.2614
current batch: 8000
Current learning rate: 0.002
Curre

...

current batch: 0
Current learning rate: 0.002
Current Batch [1] 
[Validation] Validation Loss: -298.8882, Acc: 0.7472, SE: 0.0000, SP: 0.7966, PC: 0.0000, F1: 0.0000, JS: 0.0000, DC: 0.0349, DC_RR: 0.3300
Best AttU_Net model score : 0.3300
current batch: 2000
Current learning rate: 0.002
Current Batch [2001] 
[Validation] Validation Loss: -1273.5360, Acc: 0.7393, SE: 0.4136, SP: 0.7568, PC: 0.3592, F1: 0.3017, JS: 0.2268, DC: 0.3151, DC_RR: 0.4981
Best AttU_Net model score : 0.4981
current batch: 4000
Current learning rate: 0.002
Current Batch [4001] 
[Validation] Validation Loss: -1690.7242, Acc: 0.7579, SE: 0.3868, SP: 0.7780, PC: 0.4664, F1: 0.3539, JS: 0.2728, DC: 0.3671, DC_RR: 0.5337
Best AttU_Net model score : 0.5337
current batch: 6000
Current learning rate: 0.002
Current Batch [6001] 
[Validation] Validation Loss: -1534.4221, Acc: 0.7382, SE: 0.3191, SP: 0.7485, PC: 0.2940, F1: 0.2564, JS: 0.2015, DC: 0.2748, DC_RR: 0.4713
Validation loss stop decreasing. Adjust the learning r

/home/raphael/anaconda3/envs/LUNGpy36/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


current batch: 0
Current learning rate: 0.002
Current Batch [1] 
[Validation] Validation Loss: -301.0165, Acc: 0.7472, SE: 0.0000, SP: 0.7966, PC: 0.0000, F1: 0.0000, JS: 0.0000, DC: 0.0349, DC_RR: 0.3850
Best AttU_Net model score : 0.3850
current batch: 2000
Current learning rate: 0.002
Current Batch [2001] 
[Validation] Validation Loss: -1373.5947, Acc: 0.7472, SE: 0.3841, SP: 0.7666, PC: 0.3757, F1: 0.3199, JS: 0.2454, DC: 0.3360, DC_RR: 0.5430
Best AttU_Net model score : 0.5430
current batch: 4000
Current learning rate: 0.002
Current Batch [4001] 
[Validation] Validation Loss: -1644.7914, Acc: 0.7528, SE: 0.2281, SP: 0.7804, PC: 0.3246, F1: 0.2369, JS: 0.1843, DC: 0.2616, DC_RR: 0.5037
current batch: 6000
Current learning rate: 0.002
Current Batch [6001] 
[Validation] Validation Loss: -1123.9929, Acc: 0.7453, SE: 0.0677, SP: 0.7857, PC: 0.1182, F1: 0.0775, JS: 0.0613, DC: 0.1087, DC_RR: 0.4226
Validation loss stop decreasing. Adjust the learning rate to 0.001.
current batch: 8000
C

/home/raphael/anaconda3/envs/LUNGpy36/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


current batch: 0
Current learning rate: 0.002
Current Batch [1] 
[Validation] Validation Loss: -302.7178, Acc: 0.2528, SE: 1.0000, SP: 0.0000, PC: 0.2528, F1: 0.2907, JS: 0.2528, DC: 0.2914, DC_RR: 0.1454
Best AttU_Net model score : 0.1454
current batch: 2000
Current learning rate: 0.002
Current Batch [2001] 
[Validation] Validation Loss: -1121.7316, Acc: 0.7355, SE: 0.2756, SP: 0.7445, PC: 0.3317, F1: 0.2455, JS: 0.1892, DC: 0.2641, DC_RR: 0.4992
Best AttU_Net model score : 0.4992
current batch: 4000
Current learning rate: 0.002
Current Batch [4001] 
[Validation] Validation Loss: -1535.9769, Acc: 0.7462, SE: 0.3417, SP: 0.7622, PC: 0.3909, F1: 0.2981, JS: 0.2295, DC: 0.3136, DC_RR: 0.5305
Best AttU_Net model score : 0.5305
current batch: 6000
Current learning rate: 0.002
Current Batch [6001] 
[Validation] Validation Loss: -1849.0888, Acc: 0.7567, SE: 0.4057, SP: 0.7757, PC: 0.3760, F1: 0.3306, JS: 0.2590, DC: 0.3460, DC_RR: 0.5517
Best AttU_Net model score : 0.5517
current batch: 8000